# Graded Exercise 2 - Sampling with Boltzmann Machines and MCMC

## General instruction
Each question is provided in a _Markdown_ cell and should be answered in the cell(s) below. You may add new cells if needed. All figures must be generated and shown directly in this notebook. If a question demands that you write an answer, use a _Markdown_ cell, which can include latex between \\$ symbols. As an example,
\\$\vec{F}=m\vec{a}\\$
gives $\vec{F}=m\vec{a}$.

Your code should run properly if you do the following: 1) restart the kernel 2) execute all cells in order from top to bottom. Running all cells should take a reasonable time on a standard computer or on Noto (<30 min.). Note that this exercise will require longer execution time than the first graded homework.

Avoid using `for` loops whenever possible. Instead, use vectorized operations or numpy functions.

All external sources you consult must be explicitly cited, except for the official NumPy, Scipy and Matplotlib documentation, the lecture notes, and previous exercises. You are encouraged to use external sources, since every function needed in this exercise has not necessarily been seen in the previous exercises. Please also cite every person you discussed this exercise with.

Overly long or unnecessarily complicated answers will be penalized.

## Context

In 2024, Geoffrey Hinton, one of the pioneers of deep learning, was awarded the Nobel Prize in Physics for his foundational contributions to energy-based models. His work demonstrated that ideas from physics, such as energy landscapes, entropy, and thermal fluctuations, can be used to build computational models capable of learning complex patterns from data. Central to this achievement is the Boltzmann machine, a model defined primarily by an energy function over binary variables, an idea directly inspired by the Ising model in physics.

In this exercise, you will train a Boltzmann machine or variants thereof to approximately sample from a high-dimensional distribution.

In part 1-4, we investigate the performance of various energy-based models on a synthetic dataset. In part 5, we study the performance of a specific energy-based model on a real dataset.

# 1. The bars and stripes dataset

__1.1__ Load the dataset *bars_and_stripes.npy* with the numpy `load` function. Print its shape. 

In [1]:
# Your code here

__1.2__ The *bars_and_stripes.npy* dataset consists of black and white images. Using `imshow` from the matplotlib library, draw the first 10 images of the dataset. Comment briefly on the content of this dataset.

In [3]:
# Your code here

Your answer here:

From now on, we treat each image as a flattened vector 
$$\mathbf s_\mu=(s_1^\mu, s_2^\mu, ..., s_{d}^\mu),\quad s_i\in\{0, 1\}.$$
$\mu=1, ..., n$ is the sample index, and denote the whole dataset as $S$.

**1.3** Reshape the dataset to a size $n\times d$, where $n$ indicates the number of samples and $d$ the number of pixels in each image. 

In [5]:
# Your code here

We will now study various parameterized probability distribution of the form
$$
P_\theta(\mathbf s)=\frac{e^{-E_\theta(\mathbf s)}}{Z_\theta}.
$$
$\theta$ are the parameters of the distribution, that we will find to match the given data. $E_\theta$ is a function that depends on the parameters $\theta$ and $\mathbf s$. $Z_\theta$ is the normalization such that $P_\theta$ is a proper probability distribution. Distributions of this form are called *Boltzmann* or *Gibbs* distribution. Note that to model physical systems, the $E_\theta$ is the energy and is usually multiplied by a term proportional to the inverse temperature.

# 2. Independent field model

We first define the energy of the independent field model as
$$
E_{\mathbf h}(\mathbf s)=\sum_{i=1}^d h_i s_i, \quad \mathbf h\in\mathbb{R}^d
$$
where $h_i$ are scalar parameter associated to pixel $i$. We consider the Boltzmann distribution associated to this energy
$$
P_{\mathbf{h}} (\mathbf s)=\frac{1}{Z_{\mathbf h}}e^{- E_{\mathbf h}(\mathbf s)}.
$$

**2.1** Show that in this model each pixel is statistically independent of the others.


Your answer here:

**2.2** The empirical and model averages are
$$
<s_i>_{emp}=\frac{1}{n}\sum_{\mu=1}^n s_i^\mu,
$$
$$
<s_i>_{model}=\sum_{\mathbf s\in\{0,1\}^d} s_i P_{\mathbf h}(\mathbf s).
$$
Note here that $s_i^\mu$ is component $i$ of sample $\mu$, while $s_i$ is a dummy variable which we sum over.
We study this problem with maximum likelihood estimation.
We want to find $\hat{\mathbf h}$ that minimizes the negative log-likelihood $\mathcal{L}(\mathbf h)=-\sum_{\mu=1}^n \log P_{\mathbf h}(\mathbf s_\mu)$. Show that this entails finding $\hat{\mathbf h}$ such that
$$
<s_i>_{emp}=<s_i>_{model}.
$$

Note: you can reuse the expression derived in the previous question. You are not required to check that $\hat{\mathbf h}$ is a minimum, it is sufficient to impose that it is a stationary/extremal point.

Your answer here:

**2.3** Deduce that
$$
\hat h_i = \log\left(\frac{1-<s_i>_{emp}}{<s_i>_{emp}}\right).
$$

Your answer here:

**2.4** Show that 
$$
P_{\hat h_i}(s_i)=\begin{cases}
  <s_i>_{emp} & \text{if } s_i=1, \\
  1-<s_i>_{emp}  & \text{if } s_i=0
\end{cases}
$$
where $P_{\hat h_i}(s_i)$ is the marginal probability that pixel $i$ has color $s_i$.

Your answer here:

**2.5** Write a function `sample_independent` that takes as parameter the empirical mean $<s_i>_{emp}$ and *num_samples* which samples *num_samples* images $\mathbf s$.

In [7]:
# Your code here:

**2.6** Draw 5 samples with $\mathbf h=\hat{\mathbf h}$ (i.e. with the probability distribution of 2.4) and plot them as images. What do you see ? Is it expected ? Comment briefly.

In [9]:
# Your code here:

Your answer here:

**2.7** Print the empirical mean $<\mathbf{s}>_{emp}$ that you computed. What do you observe ? What does this indicate about the dataset ? 

In [11]:
# Your code here:

Your solution here:

# 3. Boltzmann Machines

The Boltzmann machine assumes the following energy function:
$$
E_{\mathbf h, J}(\mathbf s)=-\sum_i h_i s_i-\sum_{i<j} J_{ij} s_i s_j.
$$
$J\in \mathbb{R}^{d\times d}$ are symmetric couplings ($J_{ij}=J_{ji}$), and $\mathbf h$ is as before. These are the parameters that one wants to learn. Contrary to the independent field model, there is no analytical expression to obtain $\hat J$ and $\hat{\mathbf h}$, the parameters that maximize the log-likelihood of this problem. Thus, we will estimate them using gradient descent (see also lecture 12 for more details):
$$
h_i(t+1)=h_i(t)-\eta \left( <s_i>_{model} - <s_i>_{emp}\right),
$$
$$
J_{ij}(t+1)=J_{ij}(t)-\eta \left( <s_i s_j>_{model} - <s_i s_j>_{emp}\right).
$$
To obtain the quantities $<\cdot>_{model}$, we will use Monte Carlo Markov Chains (MCMC). However, running a whole MCMC for each iteration of the gradient descent is computationally demanding. Instead of initializing the chain at a random $\mathbf s$, we start $n$ chains on each data point and run only a few MCMC steps. This is called *contrastive divergence*.

We will update these short MCMC using __Gibbs__ sampling, where each spin is resampled from its conditional distribution given all the others. The transition from a state $a$ to $b$ is in this case given by
$$
P(b\rightarrow a)=\frac{e^{-E(a)}}{e^{-E(a)}+e^{-E(b)}},
$$
where $a$ and $b$ differ at most at one pixel $s_i$. In the other cases the transition probability is $0$.
Note that other sampling schemes are also possible, such as the Metropolis algorithm.

**3.1** Show that the Gibbs sampling Markov chain satisfies detailed balance for the stationary distribution given by the Boltzmann measure with energy $E_{\mathbf h, J}(\mathbf s)$

Your answer here:

**3.2** We define $\Delta E_i$ as the difference in energy between pixel $i$ being $0$ and $1$:
$$
    \Delta E_i=E_{\mathbf{h}, J}(s_i=0)-E_{\mathbf{h}, J}(s_i=1)=h_i+\sum_{j\neq i}J_{ij}s_j
$$
Using the definition of the Boltzmann measure, show that
$$
    P_{\mathbf h, J}(s_i=1 | s_{-i})=\frac{1}{1+e^{-\Delta E_i}}
$$
where $s_{-i}$ indicates all $s_j$ except for $j=i$ (i.e. we fix every pixel except pixel $i$).

Your solution here:

**3.3** Implement a function computing the sigmoid
$$
    \sigma(x)=\frac{1}{1+e^{-x}}.
$$
Make sure that the function also works if $x$ are 1D arrays.

In [13]:
# Your code here:

**3.4** We now estimate $<\cdot>_{model}$ using MCMC. Write a function that takes as input *h*, *J*, the dataset *S* and *num_steps*. The function should:
1. Initialize $\mathbf s^\mu$ in a configuration $\mu$ from the dataset. 
2. Choose a pixel $i$ uniformly at random.
3. Compute $p=\sigma(\Delta E_i)$ (Recall that $J_{ii}$ is defined to be $0$).
4. Set $s_i^\mu=1$ with probability $p$, and else $s_i^\mu=0$.
5. Repeat from step 2 *num_steps* times.

This should be done for every initial configuration $\mu$ from the dataset in a vectorized manner. Thus, your code should only contain a loop over *num_steps*.

The function returns the updated *S* that we call $S_{new}$, an array of size $n\times d$, where we recall that $n$ is the number of datapoints and that we have one Monte-Carlo chain for each datapoint.

Note: In practice, the pixels are often updated synchronously and not randomly as asked above. This speeds up the computation, but can induce unwanted behaviours such as oscillations.

In [15]:
# Your code here:

**3.5** Write a function that estimates $<s_i>_{model}$ and $<s_i s_j>_{model}$ from a given $S_{new}$, i.e. estimate the first 2 moments. You will also use this function to compute the empirical first and second moments.

In [17]:
# Your code here

**3.6** Implement the loop implementing the gradient descent:
$$
h_i(t+1)=h_i(t)-\eta \left( <s_i>_{model} - <s_i>_{emp}\right),
$$
$$
J_{ij}(t+1)=J_{ij}(t)-\eta \left( <s_i s_j>_{model} - <s_i s_j>_{emp}\right).
$$
Choose $\eta=0.5$ and do $1000$ gradient descent steps. Estimate $<\cdot>_{model}$ using your previous code, with $300$ steps. Initialize the arrays for $\mathbf h(t=0)$ and $J(t=0)$. Each element of $\mathbf h$ should be $0$, while $J$ should be symmetric matrix, with no self coupling ($J_{ii}=0$), and the rest of the elements draw independently from a gaussian with mean $0$ and standard deviation $0.01$.

After having updated $J$, you must force it to have $0$ on the diagonal and be symmetric. Indeed, this is a constraint of the BM model, and the gradient step does not guarantee that these properties are preserved.

To monitor the convergence, print the reconstruction error
$$
E_{recon}=\|S - S_{new}(t)\|^2
$$
every 50 gradient descent step. $S$ is the data, while $S_{new}(t)$ are the generated samples used to compute $<\cdot>_{model}$ at iteration $t$ of the gradient descent. $\| \cdot \|$ is the Frobenius norm.

In [19]:
# Your code here:

**3.7** Now that the model is trained, we want to see what is has learned. To this end, we must sample from the distribution $P_{\hat{\mathbf h}, \hat J}$. We do this by again running a MCMC. Reuse your code from 3.6, but this time initialize it with 25 configurations $\mathbf s$ initialized uniformly at random. Now we also want the chain to mix, so run it for $10^4$ steps.


In [21]:
# Your code here:

**3.8** Plot the obtained samples. What do you observe ? Do you find some images that contain both horizontal and vertical bars ? If yes, explain briefly why this could happen. 

In [23]:
# Your code here:

Your answer here :

**3.9** Plot the trajectory of a single chain. Initialize it randomly, and plot what it looks like after $1, 100, 500, 10^3, 10^4, 10^5$ steps. Does the pattern significantly change between step $10^4$ and step $10^5$ ?

In [25]:
# Your code here

Your answer here:

# 4. Restricted Boltzmann machines

The Boltzmann machine can struggle to capture some structure in the data. Moreover, it can be costly to train all the pairwise interaction. To mitigate these problems, the restricted Boltzmann machine (RBM) was introduced. An RBM adds a layer of hidden units $s_j^h, j=1, ..., d_H$, and only connects the hidden and visible units $s_i^V, i=1, ..., d$.

The Restricted Boltzmann machine (RBM) assumes the following energy function:
$$
E_{\mathbf h, \tilde{\mathbf{h}} , W}(\mathbf s^v, \mathbf s^h)=-\sum_{i=1}^{d} h_i s^V_i-\sum_{j=1}^{d_H}\tilde h_j s_j^H-\sum_{i=1}^d \sum_{j=1}^{d_H} W_{ij} s_i^V s^H_j.
$$
$\mathbf s^V \in \{0,1\}^d$ are the visible units (the image), $\mathbf s^H\in\{0,1\}^{d_H}$ are the hidden units, $W\in \mathbb{R}^{d\times d_H}$ is the weight matrix connecting visible and hidden units, and $\mathbf h\in \mathbb{R}^{d}$ and $\tilde{\mathbf{h}}\in \mathbb{R}^{d_H}$ are the visible and hidden biases (or external fields) vectors.

Because there are no intra-layer connections, the conditional probabilities factorize nicely. This allows to perform __Block Gibbs Sampling__, updating all hidden units at once, then all visible units at once:
$$
P_{\mathbf h, \tilde{\mathbf{h}}, W}(s_j^H=1| \mathbf s^V)=\sigma\left(\tilde h_j+\sum_{i=1}^{d}W_{ij}s_i^V\right),
$$
$$
P_{\mathbf h, \tilde{\mathbf{h}}, W}(s_i^V=1| \mathbf s^H)=\sigma\left(h_i+\sum_{j=1}^{d_H}W_{ij}s_j^H\right).
$$

**4.1** Implement a function performing one step of the Block Gibbs Sampling. It should take the current visible state $\mathbf s^V$, and parameters $W$, $\mathbf h$ and $\tilde{\mathbf{h}}$ as inputs. Then
1. Compute the probabilities for hidden units given $\mathbf s^V$.
2. Sample the hidden state $\mathbf s^H$ from these probabilities.
3. Compute the probabilities for the visible units given this new $\mathbf s^H$.
4. Sample a new visible state $\mathbf s^V$.
The function should return both the new visible (sampled in 4) and hidden state (sampled in 2).

Note: we could continue the chain by again sampling the hidden variables with the new visible variables, etc. Doing this $k$ times is refered to CD-k (contrastive divergence with k steps). For now we keep $k=1$, but larger $k$ will be used in the next exercise.

In [27]:
# Your code here:

**4.2** The gradient descent to obtain $\hat W, \hat{\mathbf{h}}, \hat{\tilde{\mathbf{h}}}$ that maximize the likelihood in this setting is given by
$$
W_{ij}(t+1)=W_{ij}(t)-\eta(<s_i^V s_j^H>_{model}-<s_i^V s_j^H>_{emp}),
$$
$$
h_{i}(t+1)=h_{i}(t)-\eta(<s_i^V>_{model}-<s_i^V>_{emp}),
$$
$$
\tilde h_{j}(t+1)=\tilde h_{j}(t)-\eta(<s_j^H>_{model}-<s_j^H>_{emp}).
$$
Initialize $W$ with small random noise, and set the biases $\mathbf h$ and $\tilde{\mathbf{h}}$ to zeros. Set $d_H=100$. Then, run GD for $1000$ steps with $\eta=0.5$:
1. Set $\mathbf s^V$ to be a sample of the dataset.
2. Compute the probability $P_{\mathbf h, \tilde{\mathbf{h}}, W}(\mathbf s^H_j=1| \mathbf s^V)$ for each hidden unit, and sample $\mathbf s^H$ from these probabilities. $S^H$ designates the concatenation of $\mathbf s^H$ for every sample of the dataset, and is an array of size $n\times d_H$.
3. Do this for every sample of the dataset in a vectorized form, and compute $<W_{ij}>_{emp}\approx \frac{1}{n}S^T S^H$ as well as $<s_i^V>_{emp}$ and $<s_j^H>_{emp}$.
4. Compute the probability $P_{\mathbf h, \tilde{\mathbf{h}}, W}(\mathbf s^V_i=1| \mathbf s^H)$ for each visible unit, where $\mathbf s^H$ is computed in step 2. Sample $\mathbf s^V_{new}$ from this probability.
5. Compute the probability $P_{\mathbf h, \tilde{\mathbf{h}}, W}(\mathbf s^H_i=1| \mathbf s^V_{new})$ for each hidden unit, where $\mathbf s^V_{new}$ is computed in step 4. Sample $\mathbf s^H_{new}$ from this probability.
6. Do this for every sample of the dataset in a vectorized form, and estimate $<W_{ij}>_{model}\approx \frac{1}{n}S_{new}^T S^H_{new}$ as well as $<s_i^V>_{model}$ and $<s_j^H>_{model}$. $S_{new}$ designates the concatenation of $\mathbf s^V_{new}$ for every sample of the dataset and is of size $n\times d_H$, and similarly for $S^H_{new}$ which is of size $n\times d$.
7. Perform the gradient step.
To monitor the convergence, again print the reconstruction error
$$
E_{recon}=\|S - S_{new}(t)\|^2
$$
every 50 gradient descent step. $S$ is the data, while $S_{new}(t)$ are the generated samples used to compute $<\cdot>_{model}$ at iteration $t$ of the gradient descent. $\| \cdot \|$ is the Frobenius norm. 

Note: We do only one step of block Gibbs sampling. This is mainly for computational reasons, and has been shown to already yield good results. However, to estimate the $<\cdot>_{model}$ averages accurately, this chain should be run until equilibration.

Hint: you can use your previous implementation of the block Gibbs sampler to sample $S^H$ and $S_{new}$.


In [29]:
# Your code here:

**4.3** Generate new samples by running the Block Gibbs sampler for $10^4$ steps using the learnt parameters. Generate $25$ new samples and plot them. What do you observe ? Do you generate images with both vertical and horizontal lines ?

In [31]:
# Your code here:

Your answer here:

**4.4** Reproduce point 4.2 and 4.3 with only 10 hidden units. What do you observe ? Give a brief possible explanation.

In [ ]:
# Your code here:

Your answer here:

# 5. Restricted Boltzmann machines on real data

**5.1** Load the dataset *mnist.npy* using the numpy `load` function. It contains black and white $28\times 28$ images of hand-drawn digits 6 and 9. These images are taken from the MNIST dataset. Load the labels *mnist_labels.npy*, containing the labels (6 or 9) of each sample. Plot the first 5 images of the dataset, with as title the label of each image.

In [34]:
# Your code here

**5.2** Flatten the data. Using numpy's built-in functions, compute the first 2 principal components of the dataset (i.e. the two right singular vectors with largest singular values). Make sure they are normalized with respect to the euclidian norm.

Note: for this exercise, do not normalize or center the data. You can do the full singular value decomposition (even though we only need the 2 leading singular vectors). You can use numpy functions to compute this decomposition.

In [36]:
# Your code here

**5.3** Project each datapoint on the first 2 principal components (as an example, see figure 2 from lecture 2). Plot the resulting points, coloring them in different colors for each class, i.e. one color for the 6s and another color for the 9s. What do you notice ?

In [38]:
# Your code here

Your answer here:

**5.4** In the previous exercise, we used contrastive divergence with one step (CD-1), i.e. 1 step of Gibbs sampling. Implement the training using k-CD. This means that instead of computing $S^V_{new}$ and $S^H_{new}$ in only one step, you should repeat this procedure $k$ times.

Train the model on the MNIST data for $k=5$, $\eta=0.5$ and $200$ gradient descent steps. Initialize the visible and hidden biases/fields at $0$, and the coupling between the visible and hidden units to a small noise. Use $200$ hidden units. You can reuse or copy-paste previous code if needed.

Note: executing this part of the code can take ~15-20 minutes.

In [ ]:
# Your code here

__5.5__ Write a function computing the energy $E$, given the values of visible and hidden units, and parameters $W$, $\mathbf{h}$ and $\tilde{\mathbf{h}}$.

In [ ]:
# Your code here:

__5.6__ We now generate a new sample. Initialize it uniformly at random with $0$ and $1$ with $50\%$ probability, and plot how it looks like after $k=1, 10, 100, 500, 1000$ steps of CD. Additionally, compute the energy at each step and save it in a list or array.

In [ ]:
# Your code here:

__5.7__ Plot the energy computed previously as a function of the number of steps.

In [ ]:
# Your code here:

__5.8__ Generate new samples by running the Block Gibbs sampler for $1000$ steps using the learnt parameters. Generate $500$ new samples and plot them the first $25$ obtained samples.

In [ ]:
# Your code here

__5.9__ Project the obtained samples on the 2 principal components obtained before and plot them. Also plot the projection of the trained samples as before. 

In [ ]:
# Your code here